# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

In [2]:
import os
print(os.getcwd())


/Users/lizyeo/DSI/scaling_to_production/02_activities/assignments


# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [3]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
file_path = '../../05_src/data/adult/adult.data'

adult_dt = (pd.read_csv(file_path, header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [6]:
adult_dt.shape

(32561, 15)

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [5]:
from sklearn.model_selection import train_test_split

X = adult_dt.drop('income', axis=1)
Y = adult_dt['income']

#split the data into training and testing set with a 70-30 % split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.30, random_state=42)

#output the shape
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape:{X_test.shape}")
print(f"Y_train shape:{Y_train.shape}")
print(f"Y_test shape:{Y_test.shape}")

X_train shape: (22792, 14)
X_test shape:(9769, 14)
Y_train shape:(22792,)
Y_test shape:(9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*The 'random state" parameter is used in the splitting function is to set the seed for the random number generator. It ensures that the random process used to split the test and training sets are reproducible*

*It is useful as it maintains reproducibility, consistency and validation in data science experiments.*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [9]:
#output the adult.dt columns
adult_dt.columns
adult_dt.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [42]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB

#define numerical and categorical columns
numerical_cols = ['age', 'fnlwgt','education-num','capital-gain','capital-loss','hours-per-week']
categorical_cols = ['workclass', 'education','marital-status','occupation','relationship','race','sex','native-country']

#create the numerical pipeline
pipe_num_transforms = Pipeline([
    ('imputer',KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler',RobustScaler())
    ])

#create the categorical pipeline
pipe_cat_transforms = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore',drop='if_binary'))
])

#combine them into a column transformer
ctransform_simple = ColumnTransformer([
  ('numeric', pipe_num_transforms, numerical_cols),
  ('categorical', pipe_cat_transforms, categorical_cols)
])

#Main pipeline that incls preprocessing and the model
pipe_simple = Pipeline([
    ('preprocess', ctransform_simple)
    
])

pipe_simple

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])]))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [43]:
#create the main pipeline that inclds preprocessing and the RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

pipe_simple = Pipeline([
  ('preprocess', ctransform_simple),
  ('classifier', RandomForestClassifier())  
])

pipe_simple

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [35]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score


X = adult_dt.drop(columns='income')
Y = adult_dt['income']
scoring = ['neg_log_loss', 'roc_auc', 'accuracy','balanced_accuracy']

res_simple_dict = cross_validate(pipe_simple, X_train, Y_train, cv=5, scoring= scoring)
res_simple = pd.DataFrame(res_simple_dict).assign(experiment=1)
res_simple

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,fit_time,score_time,test_neg_log_loss,test_roc_auc,test_accuracy,test_balanced_accuracy,experiment
0,3.497542,0.131134,-0.335551,0.905470,0.851941,0.777504,1
1,3.432500,0.121509,-0.375933,0.901122,0.844703,0.765658,1
2,3.427650,0.121060,-0.361852,0.901696,0.853444,0.776505,1
3,3.464378,0.121576,-0.349926,0.907079,0.861343,0.785103,1
4,3.425965,0.120209,-0.373476,0.902413,0.852786,0.771456,1


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [36]:
res_simple_sorted = res_simple.sort_values(by='test_neg_log_loss')
res_simple_sorted

,fit_time,score_time,test_neg_log_loss,test_roc_auc,test_accuracy,test_balanced_accuracy,experiment
1,3.432500,0.121509,-0.375933,0.901122,0.844703,0.765658,1
4,3.425965,0.120209,-0.373476,0.902413,0.852786,0.771456,1
2,3.427650,0.121060,-0.361852,0.901696,0.853444,0.776505,1
3,3.464378,0.121576,-0.349926,0.907079,0.861343,0.785103,1
0,3.497542,0.131134,-0.335551,0.905470,0.851941,0.777504,1


Calculate the mean of each metric. 

In [37]:
mean_metrics = res_simple.mean()
mean_metrics

fit_time                  3.449607
score_time                0.123098
test_neg_log_loss        -0.359348
test_roc_auc              0.903556
test_accuracy             0.852843
test_balanced_accuracy    0.775245
experiment                1.000000
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [45]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
pipe_simple.fit(X_train,Y_train)

Y_pred_proba = pipe_simple.predict_proba(X_test)
roc_auc = roc_auc_score(Y_test, Y_pred_proba[:,1])
log_loss = log_loss(Y_test, Y_pred_proba)
accuracy = accuracy_score(Y_test, pipe_simple.predict(X_test))
balanced_accuracy = balanced_accuracy_score(Y_test, pipe_simple.predict(X_test))

#display results as a dict
results_dict = {
    'neg_log_loss': log_loss,
    'roc_auc':roc_auc,
    'accuracy':accuracy,
    'balanced_accuracy':balanced_accuracy
}

print(results_dict)

{'neg_log_loss': 0.3725893827068504, 'roc_auc': 0.9060456482981065, 'accuracy': 0.8608935974205435, 'balanced_accuracy': 0.7852438380696667}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Recoding the target variable 'income' to 0 or 1 is convenient as it makes it compatible with Scikit-learn and simplifies evaluation metrics when the target variable is a numerical)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.